# <p style="text-align: center;">Harmonizing Multi-Site Structural MRI Datasets</p>

### <p style="text-align: center;">David MacDonald</p>

# Plan
- Start simple.
- Add to the presentation every day.
- Look at other presentations every day. Brainhack ones would be a good start.
- Make it better over the course of a week.
- For now, can use canva for graphics.
- OR, LEARN to use GitPitch, which has more layout tools, I think. It was pretty cool.

- Try to describe my workflow using freepik and/or canva for images, icons, arrows, etc.
- Get icons for all of the tools that I'm using

## My Background

* git and github?
* bash
* programming (learning R, C++ a *long* time ago)
* HPC
* Excel, SPSS

# Goals

## Primary Goal: Learn how to use ComBat to harmonize multi-site sMRI data

After the first week, my goals changed.

I still want to use ComBat for my multi-site data, but now I also want to:
* Learn how to use the Python ecosystem for data analysis and visualization
* Learn how to use git and GitHub properly, and use them for collaboration and project management
    * Contribute to an open source project using the Github issue system, fork, and pull request
* Improve my science communication skills, through:
    * Learning how to create interactive representations of data
    * Using new tools to improve presentations
    * Using jupyter notebooks to package code, data visualization, and narrative together